# 클래스 공부 4단계
> 프린트 가로채기 `__str__`, `__repr__` (파이썬의 비밀2,3)

## Contents
> 1. motivating example
> 2. `__str__`, 파이썬의 비밀2
> 3. `__repr__`, 파이썬의 비밀3
> 4. 주피터 노트북의 비밀 (`_repr_html_`), `__repr__`와 `__str__`의 우선적용 순위

## imports

In [11]:
import numpy as np

## 01. Motivating Example

### `-` 가위바위보

#### class1 review

In [14]:
# class1 hw's review
class RPC:
    def throw(self):
        print(np.random.choice(['가위','바위','보']))

In [16]:
a = RPC()

In [22]:
a.throw()

가위


#### 가위,보 GAME

[가위, 바위, 보] 말고 [가위, 보] 혹은 [바위, 보] 처럼 정해진 케이스가 아닌 입력으로 받고 싶을 수도 있다.

In [34]:
class RPC:
    def throw(self, candidate):
        print(np.random.choice(candidate))

In [36]:
a = RPC()

In [41]:
# throw(a, ['가위','바위','보'])
a.throw(['가위','바위','보'])

보


In [57]:
a.throw(['가위', '보']) # 보, 가위만.

가위


#### 가위바위보1

In [114]:
class RPC:
    def __init__(self, candidate = ['가위', '바위', '보']):
        self.candidate = candidate
    def throw(self):
        print(np.random.choice(self.candidate))

In [115]:
a = RPC() # __init__ 는 암묵적으로 실행

In [116]:
a.throw()

보


#### 가위바위보2 (풀어서)

위의 코드 3줄과 동일한 코드이며, 풀어써보면 다음과 같다.

In [72]:
class RPC2:
    pass

In [76]:
b = RPC2() # 아무것도 없음..

In [77]:
def initt(b, candidate = ['가위','바위','보']):
    b.candidate = candidate

In [79]:
initt(b)

In [82]:
# 던져서 화면에 보여주는 과정까지 추가
def throw(b):
    print(np.random.choice(b.candidate))

In [83]:
throw(b)

보


#### 가위바위보2 (재조합)

풀어쓴 코드를 조합해보면?

In [84]:
class RPC2:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
    def throw(self):
        print(np.random.choice(self.candidate))

In [85]:
b = RPC2()

In [89]:
b.candidate

['가위', '바위', '보']

In [87]:
b.throw()

가위


#### throw = choose + show?

생각해보니까 throw는 choose + show의 결합인 것 같다.

In [124]:
class RPC: ## 시점1
    def __init__(self, candidate = ['가위', '바위', '보']):
        self.candidate = candidate
    def choose(self):
        self.actions = np.random.choice(self.candidate)
    def show(self):
        print(self.actions)

In [125]:
a = RPC()  ## 시점2

In [126]:
a.actions ## 시점3 (지금은 정의되지 않음, choose를 해야함)

AttributeError: 'RPC' object has no attribute 'actions'

In [127]:
a.choose() # 뭔가 선택했겠지?    ## 시점4

In [128]:
a.actions # 바위를 선택했구만     ## 시점5 

'바위'

In [129]:
a.show()   ## 시점6

바위


#### **보충학습**: 시점별 분석 (throw + show)
위와 같은 코드입니다.

In [101]:
class _RPC:  ## 시점1 
    pass  # <-- 이렇게하면 아무 기능이 없는 비어있는 클래스가 정의된다.

In [102]:
_a  = _RPC()  ## 시점2

def _init(_a, candidate = ['가위','바위','보']):
    _a.candidate = candidate
    
_init(_a)

In [105]:
_a.actions ## 시점3

AttributeError: '_RPC' object has no attribute 'actions'

In [108]:
# choose 선언      ## 시점4
def _choose(_a):
    _a.actions = np.random.choice(_a.candidate)
_choose(_a)

In [109]:
_a.actions  ## 시점5

'바위'

In [110]:
# show 선언    ## 시점6
def _show(_a):
    print(_a.actions)
_show(_a)

바위


### `-` 인스턴스 $b$ 추가

또 다른 인스턴스 b를 만들자. b는 가위만 낼 수 있다.

```
class RPC: ## 시점1
    def __init__(self, candidate = ['가위', '바위', '보']):
        self.candidate = candidate
    def choose(self):
        self.actions = np.random.choice(self.candidate)
    def show(self):
        print(self.actions)
        
```
        

In [131]:
b = RPC()

In [132]:
b.candidate

['가위', '바위', '보']

- 아무것도 없으면 b의 candidate이 가위, 가위, 보로 들어감

In [133]:
b = RPC(['가위']) # 가위만 포함된 리스트 전달

In [134]:
b.candidate

['가위']

In [135]:
b.choose()
b.show()

가위


`-` a, b의 선택들을 모아서 기록하고 싶다.

In [159]:
class RPC: ## 시점1
    def __init__(self, candidate = ['가위', '바위', '보']):
        self.candidate = candidate
        self.actions = list() ## 추가
    def choose(self):
        self.actions.append(np.random.choice(self.candidate)) ## 추가
    def show(self):
        print(self.actions)

In [160]:
a = RPC()
b = RPC(['가위'])

In [161]:
np.random.seed(123)
for i in range(5):
    a.choose()
    a.show()

['보']
['보', '바위']
['보', '바위', '보']
['보', '바위', '보', '보']
['보', '바위', '보', '보', '가위']


- `show()` 지난 히스토리까지 다 나오니까 보기 좀 불편하댜

In [162]:
np.random.seed(123)
class RPC: ## 시점1
    def __init__(self, candidate = ['가위', '바위', '보']):
        self.candidate = candidate
        self.actions = list() ## 추가
    def choose(self):
        self.actions.append(np.random.choice(self.candidate)) ## 추가
    def show(self):
        print(self.actions[-1]) ### 추추가

In [163]:
a = RPC()
b = RPC(['가위'])

In [164]:
for i in range(5):
    a.choose()
    a.show()

보
바위
보
보
가위


In [165]:
a.actions

['보', '바위', '보', '보', '가위']

In [166]:
for i in range(5):
    b.choose()
    b.show()

가위
가위
가위
가위
가위


In [167]:
b.actions

['가위', '가위', '가위', '가위', '가위']

In [168]:
a.candidate, a.actions # (낼 수 있는 패, 내가 낸 패)

(['가위', '바위', '보'], ['보', '바위', '보', '보', '가위'])

In [172]:
b.candidate, b.actions # (낼 수 있는 패, 내가 낸 패)

(['가위'], ['가위', '가위', '가위', '가위', '가위'])

`-` info라는 함수를 만들어서 a의 오브젝트가 가지고 있는 정보를 모두 보도록 하자.

(예비학습) 문자열 `\n` 이 포함된다면?

In [175]:
'클래스\n어렵네..'

'클래스\n어렵네..'

In [176]:
print('클래스\n어렵네..')

클래스
어렵네..


예비학습 끝

In [186]:
class RPC:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])      
    def info(self):
        print('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))

In [187]:
a = RPC()
b = RPC(['가위'])

In [188]:
for i in range(5):
    a.choose()
    a.show()

바위
가위
보
가위
바위


In [189]:
for i in range(5):
    b.choose()
    b.show()

가위
가위
가위
가위
가위


In [190]:
a.info()

낼 수 있는 패: ['가위', '바위', '보']
기록: ['바위', '가위', '보', '가위', '바위']


In [191]:
b.info()

낼 수 있는 패: ['가위']
기록: ['가위', '가위', '가위', '가위', '가위']


`-` 만들고보니까 info와 print의 기능이 거의 비슷함 $\to$ print(a)를 하면 a.info()와 동일한 효과를 내도록 만들 수 있을까?

`-` 말도 안되는 소리같다. 왜?
- 안될것 같은 이유1: print는 파이썬 내장기능, 내장기능을 우리가 맘대로 커스터마이징해서 쓰기는 어려울 것 같다.
- 안될 것 같은 이유2: 이유1이 해결된다 해도 문제다. 그럼 지금까지 우리가 사용했던 수 많은 print()의 결과는 어떻게 되는가?

**결론은 가능하다**

`-` 그런데 a의 자료형(RPC 자료형)에 해당하는 오브젝트에 한정하여 print를 수정하는 방법이 가능하다면? (그럼 다른 오브젝트들은 수정된 print에 영향을 받지 않음)

## 02. `__str__`

`-` 관찰1: 현재 print(a)의 결과는 아래와 같다.

In [192]:
print(a)

- a는 RPC클래스에서 만든 오브젝트이며 a가 저장된 메모리 주소는 0x7faaa7500850라는 의미

`-` 관찰2: a에는 `__str__` 이 있다.

In [193]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'actions',
 'candidate',
 'choose',
 'info',
 'show']

In [196]:
set(dir(a)) & {'__str__'}

{'__str__'}

이것을 함수처럼 사용하니까 아래와 같다.

In [194]:
a.__str__

<method-wrapper '__str__' of RPC object at 0x7faaa7500850>

In [195]:
a.__str__() # 클래스 안에 있는 메소드, 문자열 리턴

'<__main__.RPC object at 0x7faaa7500850>'

In [197]:
print(a.__str__()) # 이거 print(a)를 실행한 결과와 같다?

In [198]:
print(a)

`-` 생각: 만약에 내가 `a.__str__()` 라는 함수를 재정의 하여 리턴값을 '너는 해킹당했다'로 바꾸게 되면 print(a)해서 나오는 결과는 어떻게 될까? (약간 해커같죠)

(예비학습) 함수 덮어씌우기

In [199]:
def f():
    print('asdf')

In [200]:
f()

asdf


In [201]:
def f():
    print('guebin hahaha')

In [202]:
f()

guebin hahaha


이런식으로 함수가 이미 정의되어 있더라도, 내가 나중에 덮어씌우면 그 함수의 기능을 다시 정의한다.

(해킹시작)

In [212]:
class RPC:
    def __init__(self, candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return '너는 해킹당했다'
    def info(self):
        print('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))

In [213]:
a = RPC()

In [214]:
print(a)

너는 해킹당했다


`-` `__str__` 의 리턴값을 info에서 타이핑했던 문자열로 재정의한다면?

In [215]:
class RPC:
    def __init__(self, candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    # def info(self):
    #     print('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))
    def __str__(self):
        return('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))

In [220]:
a = RPC()

In [221]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [222]:
a.choose()
a.show()

바위


In [223]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: ['바위']


In [224]:
a.choose()
a.show()

가위


In [225]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: ['바위', '가위']


### 파이썬의 비밀2

`-` `print(a)` 와 `print(a.__str__())` 는 같은 문법이다.

`-` 참고로 `a.__str__()` 와 `str(a)` 도 같은 방법이다.

In [226]:
str(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: ['바위', '가위']"

In [227]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: ['바위', '가위']"

`-` 지금까지 우리가 썼던 기능을 확인!

(예제1)

In [228]:
a = [1,2,3]

In [229]:
print(a)

[1, 2, 3]


In [230]:
a.__str__()

'[1, 2, 3]'

In [231]:
str(a)

'[1, 2, 3]'

(예제2)

In [232]:
a = {1,2,3}
print(a)

{1, 2, 3}


In [233]:
str(a)

'{1, 2, 3}'

In [235]:
a.__str__()

'{1, 2, 3}'

(예제3)

In [236]:
a = np.array(1)
a.shape

()

In [237]:
type(a.shape)

tuple

In [238]:
print(a.shape)

()


In [239]:
a.shape.__str__()

'()'

In [240]:
str(a.shape)

'()'

(예제4)

In [241]:
a = range(10)
print(a)

range(0, 10)


In [242]:
a.__str__()

'range(0, 10)'

(예제5)

In [243]:
a = np.arange(100).reshape(10,10)
print(a)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


In [244]:
a.__str__()

'[[ 0  1  2  3  4  5  6  7  8  9]\n [10 11 12 13 14 15 16 17 18 19]\n [20 21 22 23 24 25 26 27 28 29]\n [30 31 32 33 34 35 36 37 38 39]\n [40 41 42 43 44 45 46 47 48 49]\n [50 51 52 53 54 55 56 57 58 59]\n [60 61 62 63 64 65 66 67 68 69]\n [70 71 72 73 74 75 76 77 78 79]\n [80 81 82 83 84 85 86 87 88 89]\n [90 91 92 93 94 95 96 97 98 99]]'

## 03. `__repr__`

`-` 생각해보니까  print를 써서 우리가 원하는 정보를 확인하는건 아니였음

In [267]:
a = [1,2,3]

In [268]:
a

[1, 2, 3]

In [269]:
print(a) # print(a.__str__()) + enter ==> a + enter

[1, 2, 3]


`-`` a + 엔터`를 하면 `print(a) + 엔터`를 하는 것과 같은 효과인가?

(반례)

In [270]:
a = np.array([1,2,3,4]).reshape(2,2)

In [271]:
a

array([[1, 2],
       [3, 4]])

In [272]:
print(a)

[[1 2]
 [3 4]]


`-` `a + 엔터` 는 `print(a) + 엔터` 가 다른 경우도 있다. $\to$ 추측: 서로 다른 숨겨진 기능이 있다! $\to$ 결론: 그 기능은 `__repr__` 에 저장되어 있음. 

- `__repr__` 추가 전

In [273]:
class RPC:
    def __init__(self, candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))

In [274]:
a = RPC()

In [275]:
a

In [276]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


- `__repr__` 추가 후

In [277]:
class RPC:
    def __init__(self, candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return('낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions))

In [278]:
a = RPC()

In [279]:
a # print(a.__repr__())

낼 수 있는 패: ['가위', '바위', '보']
기록: []

`-` 그럼 우리가 지금까지 했던 것?

In [280]:
a = np.array([1,2,3])

In [281]:
a

array([1, 2, 3])

In [282]:
print(a)

[1 2 3]


In [283]:
a.__repr__()

'array([1, 2, 3])'

In [284]:
a.__str__()

'[1 2 3]'

### 파이썬의 비밀3

`-` 대화형콘솔에서 `오브젝트이름 + 엔터`를 쳐서 나오는 출력은 `__repr__`의 결과와 연관이 있다.

In [285]:
a = np.array(range(10000)).reshape(100,100)
a

array([[   0,    1,    2, ...,   97,   98,   99],
       [ 100,  101,  102, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  297,  298,  299],
       ...,
       [9700, 9701, 9702, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9897, 9898, 9899],
       [9900, 9901, 9902, ..., 9997, 9998, 9999]])

In [286]:
a.__repr__()

'array([[   0,    1,    2, ...,   97,   98,   99],\n       [ 100,  101,  102, ...,  197,  198,  199],\n       [ 200,  201,  202, ...,  297,  298,  299],\n       ...,\n       [9700, 9701, 9702, ..., 9797, 9798, 9799],\n       [9800, 9801, 9802, ..., 9897, 9898, 9899],\n       [9900, 9901, 9902, ..., 9997, 9998, 9999]])'

`-` 참고로 `a.__repr__()`은 `repr(a)`와 같다.

In [287]:
repr(a)

'array([[   0,    1,    2, ...,   97,   98,   99],\n       [ 100,  101,  102, ...,  197,  198,  199],\n       [ 200,  201,  202, ...,  297,  298,  299],\n       ...,\n       [9700, 9701, 9702, ..., 9797, 9798, 9799],\n       [9800, 9801, 9802, ..., 9897, 9898, 9899],\n       [9900, 9901, 9902, ..., 9997, 9998, 9999]])'

### 주피터노트북의 비밀(`_repr_html`)

`-` 요즘에는 IDE 발전에 따라서 `오브젝트 + 엔터` 칠 때 나오는 출력의 형태도 다양해지고 있음.

In [288]:
import pandas as pd

In [289]:
df = pd.DataFrame({'a':[1,2,3],
                   'b':[2,3,4]})

In [290]:
df

,a,b
0,1,2
1,2,3
2,3,4


- 예쁘게 나온다.

`-` 위의 결과는 `print(df.__repr__())`의 결과와 조금 다르게 나온다?

In [291]:
print(df.__repr__())

   a  b
0  1  2
1  2  3
2  3  4


`-` `print(df.__repr__())`는 예전 검은화면에서 코딩할 때 나오는 출력임

```
Python 3.10.2 | packaged by conda-forge | (main, Feb  1 2022, 19:28:35) [GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
> >> import pandas as pd 
>>> df = pd.DataFrame({'a':[1,2,3],'b':[2,3,4]})>>> df
   a  b
0  1  2
1  2  3
2  3  4
>>>
```

`-` 주피터에서는 '오브젝트이름 + 엔터' 치면 `HTML(df.__repr_html())`이 실행되고 `repr_html_()`이 정의되어 있지 않으면 `print(df.__rept__())`이 실행된다.

In [293]:
df._repr_html_()

'<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>a</th>\n      <th>b</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>3</td>\n      <td>4</td>\n    </tr>\n  </tbody>\n</table>\n</div>'

- html 코드!

In [294]:
from IPython.core.display import HTML

In [295]:
HTML(df._repr_html_())

,a,b
0,1,2
1,2,3
2,3,4


`-` 물론 `df._repr_html_()`함수가 내부적으로 있어도 html이 지원되지 않는 환경이라면 `print(df.__repr__())`이 내부적으로 수행된다.

## 04. `__repr__`와 `__str__`의 우선적용 순위

(예제1)

`-` 아래의 예제를 관찰하자.

In [296]:
class RPS:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return '낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions)

In [297]:
a = RPS()
a

낼 수 있는 패: ['가위', '바위', '보']
기록: []

In [300]:
a.__repr__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [301]:
repr(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

`-` 여기까지는 상식수준의 결과임. 이제 아래를 관찰하라.

In [302]:
print(a) # print(a.__repr__())

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [304]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [305]:
str(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

- `__str__()`은 건드린적이 없는데?

In [306]:
a.__repr__??

Signature: a.__repr__()
Docstring: Return repr(self).
Source:   
    def __repr__(self):
        return '낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions)
File:      ~/Dropbox/Quarto-Blog/posts/Python/<ipython-input-296-bcd76efb6380>
Type:      method

In [307]:
a.__str__??

Signature:      a.__str__()
Call signature: a.__str__(*args, **kwargs)
Type:           method-wrapper
String form:    <method-wrapper '__str__' of RPS object at 0x7faaa47aae20>
Docstring:      Return str(self).

- `__str__()`은 건드린 적이 없는데 $\to$ 건드린적은 없는데 기능이 바뀌어있음.

(예제2)

`-` 아래의 예제를 관찰하자.

In [3]:
class RPC:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return '낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions)

In [4]:
a = RPC()

In [5]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [6]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [7]:
a.__repr__()

'<__main__.RPC object at 0x7f8f38ca22e0>'

In [8]:
a.__str__??

Signature: a.__str__()
Docstring: Return str(self).
Source:   
    def __str__(self):
        return '낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions)
File:      ~/Dropbox/Quarto-Blog/posts/Python/<ipython-input-3-2e46ee18321f>
Type:      method

In [10]:
a.__repr__??

Signature:      a.__repr__()
Call signature: a.__repr__(*args, **kwargs)
Type:           method-wrapper
String form:    <method-wrapper '__repr__' of RPC object at 0x7f8f38ca22e0>
Docstring:      Return repr(self).

2번째 예제에서는 건드린 애만 바뀌었는데 첫번째 예제에서는 건드리지 않은 애들까지 기능이 바뀌었다.

(예제3)

In [11]:
class RPC:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return '너는 해킹당했다. 하하하'
    def __str__(self):
        return '낼 수 있는 패: {}\n기록: {}'.format(self.candidate, self.actions)

In [12]:
a = RPC()

In [13]:
a

너는 해킹당했다. 하하하

In [14]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


`-` `__str__` 와 `__repr__`을 건드리지 않고 출력결과를 바꾸고 싶다면?

In [16]:
class RPC:
    def __init__(self, candidate = ['가위','바위','보']):
        self.candidate = candidate
        self.actions = list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def _repr_html_(self):
        html_str = """
        낼 수 있는 패: {} <br/>
        기록: {}
        """
        return html_str.format(self.candidate, self.actions)

In [17]:
a = RPC()

In [18]:
str(a)

'<__main__.RPC object at 0x7f8f38bb7730>'

In [21]:
repr(a)

'<__main__.RPC object at 0x7f8f38bb7730>'

In [22]:
a

In [25]:
for i in range(5):
    a.choose()
    a.show()

보
바위
가위
바위
보


In [26]:
a

## 05. 숙제

아래의 클래스를 수정하여
```python
class RPS: 
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list() 
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def _repr_html_(self):
        html_str = """
        낼 수 있는 패: {} <br/> 
        기록: {}
        """
        return html_str.format(self.candidate,self.actions)
```
클래스에서 생성된 인스턴스의 출력결과가 아래와 같도록 하라.

```
학번: 202143052 
낼 수 있는 패: ['가위', '바위', '보']
기록: ['가위', '가위', '보', '보', '바위']
```

In [43]:
class RPS: 
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate = candidate
        self.actions = list() 
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def _repr_html_(self):
        html_str = """
        학번: {} <br/>
        낼 수 있는 패: {} <br/> 
        기록: {}
        """
        return html_str.format(202143052,self.candidate,self.actions)

In [44]:
a = RPS()

In [45]:
a

In [46]:
for i in range(5):
    a.choose()
    a.show()

보
가위
바위
바위
가위


In [47]:
a